In [1]:
from models.transformers import torch_Transformer
from src.utils import model_utils

import torch



vocab_size = 10000
d_model = 512  
n_head = 8
en_layers = 6
d_ff = 2048
max_seq_length = 600


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


tr_tf_en = torch_Transformer.TRTF_encoder(vocab_size, d_model, 
    n_head, en_layers, d_ff, max_seq_length)

tr_tf_en = tr_tf_en.to(device)
tr_tf_en = tr_tf_en.bfloat16()



print(sum(p.numel() for p in tr_tf_en.parameters())/1e9, 'B parameters in tr_tf_en')




src = torch.randint(0, vocab_size, (10, 32))
src_mask = model_utils.square_subsequent_mask(10)


src = src.to(device)

src_mask = src_mask.to(device)
src_mask = src_mask.bfloat16()


output = tr_tf_en.forward(src,src_mask)
print("TRTF_en output shape",output.shape)



d:\Disk_D\programming_software\AI\Anaconda3\envs\pettingZoo_Langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda
0.024341504 B parameters in tr_tf_en
TRTF_en output shape torch.Size([10, 32, 512])


# PTF_TRTF_encoder Pretraining

In [3]:
#import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [4]:
import json
from transformers import BertTokenizer
from src.utils import data_utils

with open('./data/RL_env/Pettingzoo_mpe/simple_adversary.json','r') as file:
    data = json.load(file)

formed_data = data_utils.json2Tpath(data)


In [5]:
import sentencepiece as spm

In [6]:
from datasets import  load_dataset

dataset = load_dataset("HuggingFaceH4/CodeAlpaca_20K")


Generating test split: 100%|██████████| 2003/2003 [00:00<00:00, 400533.54 examples/s]


In [ ]:
py_codes = dataset.filter(lambda example: example['langu'])

In [4]:
tokenized_data = []
for path, text in formed_data:
    tokens = tokenizer.encode_plus(text, add_special_tokens=True, max_length=512, truncation=True, padding="max_length", return_tensors="pt").to(device)
    tokenized_data.append((path,tokens))

In [5]:
from models.transformers import torch_Transformer
from src.utils import model_utils





vocab_size = 1000
d_model = 256  
n_head = 8
en_layers = 6
d_ff = 256
max_seq_length = 512
out_dim = 64



ptf_trtf_en = torch_Transformer.paddingT_FTRTF_encoder(vocab_size, d_model, 
    n_head, en_layers, d_ff, out_dim, max_seq_length)

ptf_trtf_en = ptf_trtf_en.to(device)
ptf_trtf_en = ptf_trtf_en.bfloat16()
ptf_trtf_en.eval()


print(sum(p.numel() for p in ptf_trtf_en.parameters())/1e9, 'B parameters in tr_tf_en')





0.002778176 B parameters in tr_tf_en


In [6]:
encoded_data = []


for path, tokens in tokenized_data:
    input_ids = tokens['input_ids']
    attention_mask = tokens['attention_mask'].to(dtype=torch.float32)
    with torch.no_grad():
        output = ptf_trtf_en(input_ids, attention_mask)
    encoded_data.append((path, output))

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`